I try to simulate a **drone** in 3D\
The body is modelled as a solid ball with center DmcD\
The four motors are at distance a from the body, also modelled as solid balls, with centers $P_i$\
The four propellers are modelled as solid discs, with centers $Pp_i$. To create lift, two propellers must rotate 'positively', two 'negatively', else the drome might rotate.\
Also the radius of the propeller is optimized. 

The objective function is $\int_0^{\text{final time}} (T_1^2 + T_2^2 + T_3^2 + T_4^2) \, dt $, where $T_i$ is the magnitude of the torque acting on the propellers i.

Alternatively $\int_0^{\text{final time}} \textit{kinetic energy} \, dt $ may be used. 

The best radius of the propellerrs, $r_P$, is also found

In [ ]:
import sympy.physics.mechanics as me
from collections import OrderedDict
import time
import numpy as np
import sympy as sm
from scipy.interpolate import CubicSpline

from opty.direct_collocation import Problem
from opty.utils import parse_free
import matplotlib.pyplot as plt
import matplotlib
from IPython.display import HTML
matplotlib.rcParams['animation.embed_limit'] = 2**128
from matplotlib.animation import FuncAnimation
print(sm.__version__)

## Timo's function
to create the objective function and its gradient.

In [ ]:
def create_objective_function(
    objective, state_symbols, input_symbols, unknown_symbols,
    N, node_time_interval=1.0, integration_method="backward euler"):
    """Creates function to evaluate the objective and objective gradient.

    Parameters
    ----------
    objective : sympy.Expr
        The objective function to be minimized, which is a function of the
        states and inputs. The objective function can contain non-nested
        indefinite integrals of time, e.g. ``Integral(f(t)**2, t)``.
    state_symbols : iterable of symbols
        The state variables.
    input_symbols : iterable of symbols
        The input variables.
    unknown_symbols : iterable of symbols
        The unknown parameters.
    N : int
        Number of collocation nodes, i.e. the number of time steps.
    node_time_interval : float
        The value of the time interval. The default is 1.0, as this term only
        appears in the objective function as a scaling factor.
    integration_method : str, optional
        The method used to integrate the system. The default is "backward
        euler".

    """
    def lambdify_function(expr, multiplication_array, take_sum):
        if take_sum:
            def integration_function(x):
                return node_time_interval * np.sum(x * multiplication_array)
        else:
            def integration_function(x):
                return node_time_interval * x * multiplication_array
        return sm.lambdify(
            (states, inputs, params), expr,
            modules=[{int_placeholder.name: integration_function}, "numpy"],
            cse=True)

    def parse_expr(expr, in_integral=False):
        if not expr.args:
            return expr
        if isinstance(expr, sm.Integral):
            if in_integral:
                raise NotImplementedError("Nested integrals are not supported.")
            if expr.limits != ((me.dynamicsymbols._t,),):
                raise NotImplementedError(
                    "Only indefinite integrals of time are supported.")
            return int_placeholder(parse_expr(expr.function, True))
        return expr.func(*(parse_expr(arg) for arg in expr.args))

    # Parse function arguments
    states = sm.ImmutableMatrix(state_symbols)
    inputs = sm.ImmutableMatrix(sort_sympy(input_symbols))
    params = sm.ImmutableMatrix(sort_sympy(unknown_symbols))
    if states.shape[1] > 1 or inputs.shape[1] > 1 or params.shape[1] > 1:
        raise ValueError(
            'The state, input, and unknown symbols must be column matrices.')
    n, q = states.shape[0], inputs.shape[0]
    i_idx, r_idx = n * N, (n + q) * N

    # Compute analytical gradient of the objective function
    objective_grad = sm.ImmutableMatrix([objective]).jacobian(
        states[:] + inputs[:] + params[:])

    # Replace the integral with a custom function
    int_placeholder = sm.Function("_IntegralFunction")
    objective = parse_expr(objective)
    objective_grad = tuple(parse_expr(objective_grad))

    # Replace zeros with an array of zeros, otherwise lambdify will return a
    # scalar zero instead of an array of zeros.
    objective_grad = tuple(
        np.zeros(N) if grad == 0 else grad for grad in objective_grad[:n + q]
        ) + tuple(objective_grad[n + q:])

    # Define evaluation functions based on the integration method
    if integration_method == "backward euler":
        obj_expr_eval = lambdify_function(
            objective, np.hstack((0, np.ones(N - 1))), True)
        obj_grad_time_expr_eval = lambdify_function(
            objective_grad[:n + q], np.hstack((0, np.ones(N - 1))), False)
        obj_grad_param_expr_eval = lambdify_function(
            objective_grad[n + q:], np.hstack((0, np.ones(N - 1))), True)
        def obj(free):
            states = free[:i_idx].reshape((n, N))
            inputs = free[i_idx:r_idx].reshape((q, N))
            return obj_expr_eval(states, inputs, free[r_idx:])

        def obj_grad(free):
            states = free[:i_idx].reshape((n, N))
            inputs = free[i_idx:r_idx].reshape((q, N))
            return np.hstack((
                *obj_grad_time_expr_eval(states, inputs, free[r_idx:]),
                obj_grad_param_expr_eval(states, inputs, free[r_idx:])
             ))

    elif integration_method == "midpoint":
        obj_expr_eval = lambdify_function(
            objective, np.ones(N - 1), True)
        obj_grad_time_expr_eval = lambdify_function(
            objective_grad[:n + q], np.hstack((0.5, np.ones(N - 2), 0.5)),
            False)
        obj_grad_param_expr_eval = lambdify_function(
            objective_grad[n + q:], np.ones(N - 1), True)
        def obj(free):
            states = free[:i_idx].reshape((n, N))
            states_mid = 0.5 * (states[:, :-1] + states[:, 1:])
            inputs = free[i_idx:r_idx].reshape((q, N))
            inputs_mid = 0.5 * (inputs[:, :-1] + inputs[:, 1:])
            return obj_expr_eval(states_mid, inputs_mid, free[r_idx:])

        def obj_grad(free):
            states = free[:i_idx].reshape((n, N))
            states_mid = 0.5 * (states[:, :-1] + states[:, 1:])
            inputs = free[i_idx:r_idx].reshape((q, N))
            inputs_mid = 0.5 * (inputs[:, :-1] + inputs[:, 1:])
            return np.hstack((
                *obj_grad_time_expr_eval(states, inputs, free[r_idx:]),
                obj_grad_param_expr_eval(states_mid, inputs_mid, free[r_idx:])
            ))

    else:
        raise NotImplementedError(
            f"Integration method '{integration_method}' is not implemented.")

    return obj, obj_grad


def sort_sympy(seq):
    """Returns a sorted list of the symbols."""
    seq = list(seq)
    try:  # symbols
        seq.sort(key=lambda x: x.name)
    except AttributeError:  # functions
        seq.sort(key=lambda x: x.__class__.__name__)
    return seq

## Kane's equations of motion

The power of the radius of the propeller used in the friction and in the lift are completely arbitrary. I just played around until I found powers, which optimize the radius inside the bound, not at either limit.\
Also, I have no idea, whether friction and lift depend linearly on the rotational speed of the propeller, I simply assumed without doing any checking\
Everything else is standard.


In [ ]:
start = time.time()
start0 = time.time()

N, Ab = sm.symbols('N Ab', cls= me.ReferenceFrame)                              # inertial frame, frame of the drone body
Ap1, Ap2, Ap3, Ap4 = sm.symbols('Ap1 Ap2 Ap3 Ap4', cls= me.ReferenceFrame)      # frame of the propellers
t = me.dynamicsymbols._t
O, DmcD, P1, P2, P3, P4 = sm.symbols('O DmcD P1 P2 P3 P4', cls= me.Point)       # origin, mass center of the drone, of the motors P1...P4
Pp1, Pp2, Pp3, Pp4 = sm.symbols('Pp1 Pp2 Pp3 Pp4', cls= me.Point)               # mass centers of the propellers
O.set_vel(N, 0)

x, y, z   = me.dynamicsymbols('x y z')                                          # position of the mass center of the drone
ux, uy, uz = me.dynamicsymbols('u_x, u_y, u_z')                                             
q1, q2, q3 = me.dynamicsymbols('q_1, q_2, q_3')                                 # angle of rotation of the drone wrt N
u1, u2, u3 = me.dynamicsymbols('u_1, u_2, u_3')         

qp1, qp2, qp3, qp4 = me.dynamicsymbols('q_p1, q_p2, q_p3, q_p4')                # angle of rotation of the propellers wrt Ab
up1, up2, up3, up4 = me.dynamicsymbols('u_p1, u_p2, u_p3, u_p4')

T1, T2, T3, T4 = me.dynamicsymbols('T_1, T_2, T_3, T_4')                        # torques applied to the propellers, control input 

mD, mM, mP, g, a, reibung = sm.symbols('m_D,m_M, m_P, g, a, reibung')           # mass of body of ther drone, mass of the motors, mass of propellers per unit radius of it , gravity, distance of the motors from the mass center, friction coefficient
rb, rM, rP, l1 = sm.symbols('r_b r_M r_P l_1')                                  # radius of the body, radius of the motors, radius of the propellers, location of the propellers 'above' the drone
reibungP = sm.symbols('reibungP')                                               # friction coefficient of the propellers

Ab.orient_body_fixed(N, (q3, q1, q2), '312')                                                   
rot = Ab.ang_vel_in(N)                                                          # needed for the kinematic equations                                                       
Ab.set_ang_vel(N, u1 * Ab.x + u2*Ab.y + u3*Ab.z)                                
rot1 = Ab.ang_vel_in(N)                                                         # needed for the kinematic equations            

Ap1.orient_axis(Ab, qp1, Ab.z)                                                  # orientation of the propellers
Ap1.set_ang_vel(Ab, up1 * Ab.z)
Ap2.orient_axis(Ab, qp2, Ab.z)
Ap2.set_ang_vel(Ab, up2 * Ab.z)
Ap3.orient_axis(Ab, qp3, Ab.z)
Ap3.set_ang_vel(Ab, up3 * Ab.z)
Ap4.orient_axis(Ab, qp4, Ab.z)
Ap4.set_ang_vel(Ab, up4 * Ab.z)

DmcD.set_pos(O, x*N.x + y*N.y + z*N.z)                                          # position of the mass center of the drone
DmcD.set_vel(N, ux*N.x + uy*N.y + uz*N.z)

P1.set_pos(DmcD, a * Ab.x)                                                      # position of the motors
P2.set_pos(DmcD, -a * Ab.x)
P3.set_pos(DmcD, a * Ab.y)
P4.set_pos(DmcD, -a * Ab.y)

P1.v2pt_theory(DmcD, N, Ab)
P2.v2pt_theory(DmcD, N, Ab)
P3.v2pt_theory(DmcD, N, Ab)
P4.v2pt_theory(DmcD, N, Ab)

Pp1.set_pos(P1, l1 * Ab.z)                                                      # position of the propellers
Pp2.set_pos(P2, l1 * Ab.z)
Pp3.set_pos(P3, l1 * Ab.z)
Pp4.set_pos(P4, l1 * Ab.z)

Pp1.v2pt_theory(P1, N, Ab)
Pp2.v2pt_theory(P2, N, Ab)
Pp3.v2pt_theory(P3, N, Ab)
Pp4.v2pt_theory(P4, N, Ab)


# Create the bodies
# drone body and motors are modelled as solid balls of of radius rb, rM and masses mD, mM
# the propellers are modelled as solid disks of radius rP and mass mP
iXXd = 2./5 * mD * rb**2
iYYd = iXXd
iZZd = iXXd
inertiaD = (me.inertia(Ab, iXXd, iYYd, iZZd))
DmcDa = me.RigidBody('DmcDa', DmcD, Ab, mD, (inertiaD, DmcD))

iXXm = 2./5 * mM * rM**2
iYYm = iXXm
iZZm = iXXm
inertiaM = (me.inertia(Ab, iXXm, iYYm, iZZm))
P1a = me.RigidBody('P1a', P1, Ab, mM, (inertiaM, P1))
P2a = me.RigidBody('P2a', P2, Ab, mM, (inertiaM, P2))
P3a = me.RigidBody('P3a', P3, Ab, mM, (inertiaM, P3))
P4a = me.RigidBody('P4a', P4, Ab, mM, (inertiaM, P4))

iXXp = 1./4. * (mP * rP) * rP**2
iYYp = iXXp
iZZp = 1./2. * (mP * rP) * rP**2
inertiaP1 = (me.inertia(Ap1, iXXp, iYYp, iZZp))
inertiaP2 = (me.inertia(Ap2, iXXp, iYYp, iZZp))
inertiaP3 = (me.inertia(Ap3, iXXp, iYYp, iZZp))
inertiaP4 = (me.inertia(Ap4, iXXp, iYYp, iZZp))
Pp1a = me.RigidBody('Pp1a', Pp1, Ap1, mP, (inertiaP1, Pp1))
Pp2a = me.RigidBody('Pp2a', Pp2, Ap2, mP, (inertiaP2, Pp2))
Pp3a = me.RigidBody('Pp3a', Pp3, Ap3, mP, (inertiaP3, Pp3))
Pp4a = me.RigidBody('Pp4a', Pp4, Ap4, mP, (inertiaP4, Pp4))

BODY = [DmcDa, P1a, P2a, P3a, P4a, Pp1a, Pp2a, Pp3a, Pp4a]

# Forces
# I assume, that the frictional forces are proportionmal to the velocity of the drone squared, and diminish with heigtht.
# The exponents I use for the radisu of the propellers, rD, for the fritction and for the lift are completely ARBITRARY.
# I simply played around, until the optimum would be somewhere between the bounds.
# I did not bother to look for 'realistic exponents, nor did I look, whether the rotational speed enters linearly, as it does here
FL = [
    (Ap1, T1 * Ab.z - reibungP * rP**(2/3) * up1 * Ab.z),                                                               # drive the propellers. the control variables.
    (Ap2, T2 * Ab.z - reibungP * rP**(2/3) * up2 * Ab.z),
    (Ap3, T3 * Ab.z - reibungP * rP**(2/3) * up3 * Ab.z),
    (Ap4, T4 * Ab.z - reibungP * rP**(2/3) * up4 * Ab.z),
    (Ab, -(T1 + T2 + T3 + T4) * Ab.z),                                                                      # moment of the propellers onto the drone
    (DmcD, -mD*g*N.z - reibung * DmcD.vel(N).magnitude()*DmcD.vel(N) / (1 + z**2)), 
    (P1,   -mM*g*N.z - reibung * P1.vel(N).magnitude()*P1.vel(N) / (1 + z**2)), 
    (P2,   -mM*g*N.z - reibung * P2.vel(N).magnitude()*P2.vel(N) / (1 + z**2)),
    (P3,   -mM*g*N.z - reibung * P3.vel(N).magnitude()*P3.vel(N) / (1 + z**2)),
    (P4,   -mM*g*N.z - reibung * P4.vel(N).magnitude()*P4.vel(N) / (1 + z**2)),
    (Pp1,  -mP*g*N.z - reibung * Pp1.vel(N).magnitude()*Pp1.vel(N) / (1 + z**2) + rP**2 * up1 * Ap1.z),      # propellers must run in different directions
    (Pp2,  -mP*g*N.z - reibung * Pp2.vel(N).magnitude()*Pp2.vel(N) / (1 + z**2) - rP**2 * up2 * Ap2.z),      # otherwise the drone should rotate
    (Pp3,  -mP*g*N.z - reibung * Pp3.vel(N).magnitude()*Pp3.vel(N) / (1 + z**2) + rP**2 * up3 * Ap3.z),
    (Pp4,  -mP*g*N.z - reibung * Pp4.vel(N).magnitude()*Pp4.vel(N) / (1 + z**2) - rP**2 * up4 * Ap4.z),
    ]                                                    

kd = sm.Matrix([
               ux - x.diff(t), uy - y.diff(t), uz - z.diff(t), 
               *[(rot-rot1).dot(uv) for uv in N],
               up1 - qp1.diff(t), up2 - qp2.diff(t), up3 - qp3.diff(t), up4 - qp4.diff(t),
               ]) 
config_constr = []
speed_constr  = []
q_ind = [x, y, z, q1, q2, q3, qp1, qp2, qp3, qp4]
q_dep = []
u_ind = [ux, uy, uz, u1, u2, u3, up1, up2, up3, up4]
u_dep = []

# Create the KanesMethod object
KM = me.KanesMethod(
                    N, 
                    q_ind=q_ind, 
                    q_dependent=q_dep,
                    u_ind=u_ind, 
                    u_dependent=u_dep, 
                    kd_eqs=kd,
                    configuration_constraints=config_constr,
                    velocity_constraints=speed_constr,
                    )
(fr, frstar) = KM.kanes_equations(BODY, FL)
EOM = kd.col_join(fr + frstar) 

print('EOM shape', EOM.shape, '\n')
print('EOM DS', me.find_dynamicsymbols(EOM), '\n')
print('EOM FS', EOM.free_symbols, '\n')
print(f'EOMs contain {sm.count_ops(EOM)} operations, {sm.count_ops(sm.cse(EOM))} after cse', '\n')

qL = [wert for wert in KM.q] + [wert for wert in KM.u]
print('sequence of states:', qL)

## opty
Set up the machinery for **opty** and **solve** the optimization: Get DmcD from a predetermined point to another predetermined point, with predetermined angles $q_i$ and final speeds.\
For some reason, final drone speeds $u_x = u_y = 0$ are difficult, while, say, $u_x = u_y = 2.5$ pose no problems. So I iterate to $u_x = u_y \approx 0$

In [ ]:
state_symbols     = tuple((wert for wert in qL))
laenge            = len(state_symbols)
constant_symbols  = tuple((mD, mM, mP, g, a, reibung, rb, rM))
specified_symbols = ((T1, T2, T3, T4))
unknown_symbols   = [rP] 
 
#======================================================================================================
methode   = 'backward euler'    # The integration method to use. backward euler or midpoint are the choices
opty_info = True                # If True, the optimizer will print some information about the optimization process
kinetik   = True                # If True, the kinetic energy will be used as objective, otherwise the squares of the torques
#======================================================================================================

duration  = 10.                                             # The duration of the simulation.
num_nodes = 100                                             # The number of nodes to use in the direct collocation problem.
t0, tf    = 0., duration                                    # The initial and final times
t_int     = duration / 2.                                   # The intermediate time

initial_guess = np.ones((len(state_symbols) + len(specified_symbols)) * num_nodes + len(unknown_symbols))

interval_value = duration / (num_nodes - 1)

# Specify the known system parameters.
par_map = OrderedDict()
par_map[mD]       = 5.0                                     # Mass of the drone body
par_map[mM]       = 0.5                                     # Mass of the motors
par_map[mP]       = 0.1                                     # Mass of the propellers
par_map[g]        = 9.81                                    # Acceleration due to gravity
par_map[a]        = 3.0                                     # Distance of the mass center of the drone from P_i
#======================================================================================================
#======================================================================================================
par_map[reibung]  = 0.                                      # Friction coefficient.
par_map[reibungP] = 0.25                                    # Friction coefficient of the propellers
#======================================================================================================
#======================================================================================================
par_map[rb]       = 2.0                                     # Radius of the body
par_map[rM]       = 0.5                                     # Radius of the motors
par_map[l1]       = 0.5                                     # Distance of the propellers from the mass center

# create_ogjective_function needs the objective where the constant symbols have been replaced by their values
kin_energie = sum([me.kinetic_energy(N, body) for body in BODY]).subs({key: par_map[key] for key in par_map.keys()}) # The kinetic energy of the system should be minimal

if kinetik == True:
    objective = sm.Integral(kin_energie, t)
else:
    objective = sm.Integral(T1**2 + T2**2 + T3**2 + T4**2, t)   # minimal power


# Specify the objective function and it's gradient.
obj, obj_grad = create_objective_function(
    objective, 
    state_symbols, 
    specified_symbols, 
    unknown_symbols,
    num_nodes, 
    interval_value, 
    methode)


#====================================================================================
initial_state_constraints = {
                            x: 0,
                            y: 0.,
                            z: 0,
                            q1: 0,
                            q2: 0,
                            q3: 0,
                            qp1: 0,
                            qp2: 0,
                            qp3: 0,
                            qp4: 0,
                            ux: 0.,
                            uy: 0.,
                            uz: 0.,
                            u1: 0.,
                            u2: 0.,
                            u3: 0.,
                            up1: 0.,
                            up2: 0.,
                            up3: 0.,
                            up4: 0.,
                            } 


intermediate_state_constraints = {
                                 x: 50.,
                                 y: 50.,
                                 z: 75.,
                                 q1: 0.,
                                 q2: 0.,
                                 ux: 0.,   # Value has no meaning, will be set during iteration
                                 uy: 0.,   # Value has no meaning, will be set during iteration
                                 uz: 0.,   # Value has no meaning, will be set during iteration          
                                 u1: 0.,
                                 u2: 0.,                 
                                }

final_state_constraints    = {
                            x: 100,
                            y: 100.,
                            z: 100.,
                            q1: 0.,
                            q2: 0.,
                            ux: 0.,   # Value has no meaning, will be set during iteration
                            uy: 0.,   # Value has no meaning, will be set during iteration
                            uz: 0.,   # Value has no meaning, will be set during iteration
                            u1: 0.,
                            u2: 0.,
                             }    


staerke =  100.                                                 # The maximum strength of the control input
bounds = {
         T1: (-staerke, staerke),
         T2: (-staerke, staerke),
         T3: (-staerke, staerke),
         T4: (-staerke, staerke),
         z:  (0., np.inf),
         q1: (-np.pi, np.pi),
         q2: (-np.pi, np.pi),
         rP: (0.5, 5.)
         }
    
instance_constraints = (
                       *tuple(xi.subs({t: t0}) - xi_val for xi, xi_val in initial_state_constraints.items()), 
                       *tuple(xi.subs({t: t_int}) - xi_val for xi, xi_val in intermediate_state_constraints.items()),
                       *tuple(xi.subs({t: tf}) - xi_val for xi, xi_val in final_state_constraints.items()),
                       )


# Here I can iterate. This helps sometimes.
solution_list = []                        # list to store the solutions of each iteration
g_list        = []                        # list to store the values of the errors

for i in range(5): 
# for some reason, the final states ux = uY = 0 give problems. Hence I itetate towards them
    final_state_constraints[ux] = 2.500001 - (i+1)/5.
    final_state_constraints[uy] = 2.500001 - (i+1)/5.
    final_state_constraints[uz] = 2.500001 - (i+1)/5.

    intermediate_state_constraints[ux] = 2.500001 - (i+1)/5.
    intermediate_state_constraints[uy] = 2.500001 - (i+1)/5.
    intermediate_state_constraints[uz] = 2.500001 - (i+1)/5.

    par_map[reibung] = 0.5  * i                             # Friction coefficient. May be chaned during in each loop

    instance_constraints = (
                            *tuple(xi.subs({t: t0}) - xi_val for xi, xi_val in initial_state_constraints.items()), 
                            *tuple(xi.subs({t: t_int}) - xi_val for xi, xi_val in intermediate_state_constraints.items()),
                            *tuple(xi.subs({t: tf}) - xi_val for xi, xi_val in final_state_constraints.items()),
                            ) 

    prob = Problem(obj, obj_grad, EOM, state_symbols, num_nodes, interval_value,
        known_parameter_map=par_map,
        instance_constraints=instance_constraints,
        bounds=bounds,
        integration_method=methode)

    prob.add_option('max_iter', 24000)           # default is 3000
    
# Find the optimal solution.
    solution, info = prob.solve(initial_guess)
    print(f'{i+1} - th iteration')
    if opty_info:
        print('message from optimizer:', info['status_msg'])
        print('Iterations needed',len(prob.obj_value))
        print(f"objective value {info['obj_val']:.3e}")
        print(f'radius of propellers              {solution[-1]:.3f}')
#        print(f'distance of propeller from motor {solution[-1]:.3f} \n')
    prob.plot_objective_value()
    initial_guess = solution
    solution_list.append(solution)
    g_list.append(info['g']) 

Plot the **errors** in the gereralized coordinates and speeds.\
Print, how well the **final_state_constraints** are fulfilled.

In [ ]:
welche = -1                 # determines the solution of which iteration to use
fehler = g_list[welche]
anzahl = len(state_symbols)
fig, ax = plt.subplots(anzahl, 1, figsize=(10, 2.25*anzahl), sharex=True)
times = np.linspace(0.0, duration, num=num_nodes)

for i, j in enumerate(state_symbols):
    if j in final_state_constraints.keys():
        farbe1 = 'red'
    else:
        farbe1 = 'black'

    if i < laenge:
        farbe = 'b'
    else:
        farbe = 'r'
    ax[i].plot(times, fehler[i * num_nodes:(i + 1) * num_nodes], color = farbe, lw=0.5)
    ax[i].grid()
    ax[i].set_ylabel(f'${str(j)}$', color=farbe1)
ax[-1].set_xlabel('time')
ax[0]. set_title(f'Errors in the generalized coordinates and speeds \n coordinates/speeds in the final_state_constraints are red', color = 'b')

zaehler = -(len(intermediate_state_constraints) + len(final_state_constraints)) - 1
for i in intermediate_state_constraints.keys():
    zaehler += 1
    print(f'error in the intermediate state constraint {i}:   {fehler[zaehler]:.3e}')
print('\n')
for i in final_state_constraints.keys():
    zaehler += 1
    print(f'error in the final state constraint {i}:   {fehler[zaehler]:.3e}')

Plot location and speed of the drone, and also the forces acting on it.

In [ ]:
solution = solution_list[welche]
anzahl = len(state_symbols) + len(specified_symbols)
fig, ax = plt.subplots(anzahl, 1, figsize=(10, 2.5*anzahl), sharex=True)
times = np.linspace(0.0, duration, num=num_nodes)

for i, j in enumerate(state_symbols + specified_symbols):
    if j in final_state_constraints.keys():
        farbe1 = 'red'
    else:
        farbe1 = 'black'

    if i < laenge:
        farbe = 'b'
    else:
        farbe = 'r'
    ax[i].plot(times, solution[i * num_nodes:(i + 1) * num_nodes], color = farbe, lw = 0.5)
    ax[i].grid()
    ax[i].set_ylabel(f'${str(j)}$', color=farbe1)
ax[-1].set_xlabel('time')
ax[0]. set_title('Generalized coordinates and speeds', color = 'b')
ax[laenge]. set_title('Control force(s)', color = 'r');

**animation**

In [ ]:
fps = 20

# trace the paths of the drone body and of the propellers
traceD  = []
traceP1 = []
traceP2 = []
traceP3 = []
traceP4 = []

# Copied from Timo. Makes the animation much easier.
state_vals, input_vals, _ = parse_free(solution, len(state_symbols), len(specified_symbols), num_nodes)
t_arr = np.linspace(t0, tf, num_nodes)
state_sol = CubicSpline(t_arr, state_vals.T)
input_sol = CubicSpline(t_arr, input_vals.T)

qL = [*state_symbols, *specified_symbols]
par_map[a] = 15.   # I change this for better visibility
pL, pL_vals = [*constant_symbols], [par_map[const] for const in constant_symbols]

# define final points of the force arrows
f1, f2, f3, f4 = sm.symbols('f1 f2 f3 f4')                          # the forces of the propellers, will be calculated below
Ff1, Ff2, Ff3, Ff4 = sm.symbols('Ff1, Ff2, Ff3, Ff4', cls = me.Point)
Ff1.set_pos(P1, f1 * Ab.z)
Ff2.set_pos(P2, f2 * Ab.z)
Ff3.set_pos(P3, f3 * Ab.z)
Ff4.set_pos(P4, f4 * Ab.z)

coordinates = DmcD.pos_from(O).to_matrix(N)
for point in [P1, P2, P3, P4, Ff1, Ff2, Ff3, Ff4]:
    coordinates = coordinates.row_join(point.pos_from(O).to_matrix(N))
coords_lam = sm.lambdify(qL + pL + [f1, f2, f3, f4], coordinates, cse=True)

# configuration constraints seem to 'mess up ther order of the states. I need to find the indices of the states, as they are in the configuration constraints
# to keep the order of the states in the animation
idx = []
for ort in initial_state_constraints.keys():
    idx.append(qL.index(ort))

# needed to give the picture the right size
xmin, xmax = np.min(state_vals[idx[0], :]) - par_map[a], np.max(state_vals[idx[0], :]) + par_map[a]
ymin, ymax = np.min(state_vals[idx[1], :]) - par_map[a], np.max(state_vals[idx[1], :]) + par_map[a]
zmin, zmax = np.min(state_vals[idx[2], :]) - par_map[a], np.max(state_vals[idx[2], :]) + par_map[a]

def plot_3d_plane(x_min, x_max, y_min, y_max):
    # Create a meshgrid for x and y values
    x = np.linspace(x_min, x_max, 100)
    y = np.linspace(y_min, y_max, 100)
    x, y = np.meshgrid(x, y)

    # Z values are set to 0 for the plane
    z = np.zeros_like(x)
   
    # Plot the 3D plane
    ax.plot_surface(x, y, z, alpha=0.1, rstride=100, cstride=100, color='c')


fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='3d')
ax.set_xlim(xmin - 1., xmax + 1.)
ax.set_ylim(ymin - 1., ymax + 1.)
ax.set_zlim(zmin - 1., zmax + 1.)
ax.set_aspect('equal')
ebene = plot_3d_plane(xmin - 1., xmax + 1., ymin - 1., ymax + 1.)


# Initialize an empty line
line1, = ax.plot([], [], [], color='red', marker='o', markersize=10)            # the drone body
line2, = ax.plot([], [], [], color='blue', marker='o', markersize=5)            # the propeller 1
line3, = ax.plot([], [], [], color='magenta', marker='o', markersize=5)         # the propeller 2
line4, = ax.plot([], [], [], color='orange', marker='o', markersize=5)          # the propeller 3
line5, = ax.plot([], [], [], color='black', marker='o', markersize=5)           # the propeller 4
line6, = ax.plot([], [], [], color='black', lw=0.5, markersize=1)               # connecting P1 to P2
line7, = ax.plot([], [], [], color='black', lw=0.5, markersize=1)               # connecting P3 to P4

line8, = ax.plot([], [], [], color='red', lw=0.5, markersize=1)                 # trace drome body
line9, = ax.plot([], [], [], color='blue', lw=0.25, markersize=1)               # trace propeller 1
line10, = ax.plot([], [], [], color='magenta', lw=0.25, markersize=1)           # trace propeller 2
line11, = ax.plot([], [], [], color='orange', lw=0.25, markersize=1)            # trace propeller 3
line12, = ax.plot([], [], [], color='black', lw=0.25, markersize=1)             # trace propeller 4


quiver1 = ax.quiver([], [], [], [], [], [], color='r')                          # the force arrow
quiver2 = ax.quiver([], [], [], [], [], [], color='r')                          # the force arrow
quiver3 = ax.quiver([], [], [], [], [], [], color='r')                          # the force arrow
quiver4 = ax.quiver([], [], [], [], [], [], color='r')                          # the force arrow

ax.plot(initial_state_constraints[x], initial_state_constraints[y], initial_state_constraints[z], marker='o', markersize=5, color='red')                     # the starting point
ax.plot(intermediate_state_constraints[x], intermediate_state_constraints[y], intermediate_state_constraints[z], marker='o', markersize=5, color='blue')     # the intermediate point
ax.plot(final_state_constraints[x], final_state_constraints[y], final_state_constraints[z], marker='o', markersize=5, color='green')                         # the end point

# Function to update the plot for each animation frame
def update(t):
    global quiver1, quiver2, quiver3, quiver4
    message = (f'running time {t:.2f} sec \n The green arrows are the forces \n Optimal radius of the propellers is {solution[-1]:.3f}')
    ax.set_title(message, fontsize=15)

    skala = 5.0                                                               # scaling factor for the forces, to make them look 'right'. Apparently the only way to do this with quiver in 3D
    f11 = solution[-1]**2 * state_sol(t)[idx[-4]] / skala                          # the forces of the propellers
    f21 = -solution[-1]**2 * state_sol(t)[idx[-3]] / skala
    f31 = solution[-1]**2 * state_sol(t)[idx[-2]] / skala
    f41 = -solution[-1]**2 * state_sol(t)[idx[-1]] / skala

    coords = coords_lam(*state_sol(t), *input_sol(t), *pL_vals, f11, f21, f31, f41)
    line1.set_data([coords[0, 0]], [coords[1, 0]])                          # the drone body
    line1.set_3d_properties([coords[2, 0]])
    line2.set_data([coords[0, 1]], [coords[1, 1]])                          # the propeller 1
    line2.set_3d_properties([coords[2, 1]])
    line3.set_data([coords[0, 2]], [coords[1, 2]])                          # the propeller 2
    line3.set_3d_properties([coords[2, 2]])
    line4.set_data([coords[0, 3]], [coords[1, 3]])                          # the propeller 3
    line4.set_3d_properties([coords[2, 3]])
    line5.set_data([coords[0, 4]], [coords[1, 4]])                          # the propeller 4
    line5.set_3d_properties([coords[2, 4]])
    
    line6.set_data([coords[0, 1], coords[0, 2]], [coords[1, 1], coords[1, 2]])  # connecting P1 to P2
    line7.set_data([coords[0, 3], coords[0, 4]], [coords[1, 3], coords[1, 4]])  # connecting P3 to P4
    line6.set_3d_properties([coords[2, 1], coords[2, 2]])
    line7.set_3d_properties([coords[2, 3], coords[2, 4]])

    quiver1.remove()
    quiver2.remove()
    quiver3.remove()
    quiver4.remove()
    quiver1 = ax.quiver(coords[0, 1], coords[1, 1], coords[2, 1], coords[0, 5] - coords[0, 1], coords[1, 5] - coords[1, 1],coords[2, 5] - coords[2, 1], color='g')
    quiver2 = ax.quiver(coords[0, 2], coords[1, 2], coords[2, 2], coords[0, 6] - coords[0, 2], coords[1, 6] - coords[1, 2],coords[2, 6] - coords[2, 2], color='g')
    quiver3 = ax.quiver(coords[0, 3], coords[1, 3], coords[2, 3], coords[0, 7] - coords[0, 3], coords[1, 7] - coords[1, 3],coords[2, 7] - coords[2, 3], color='g')
    quiver4 = ax.quiver(coords[0, 4], coords[1, 4], coords[2, 4], coords[0, 8] - coords[0, 4], coords[1, 8] - coords[1, 4],coords[2, 8] - coords[2, 4], color='g')
#    quiver3 = ax.quiver(coords[0, idx[3]], coords[1, idx[3]], coords[2, idx[3]], coords[0, idx[7]] - coords[0, idx[3]], coords[1, idx[7]] - coords[1, idx[3]],coords[2, idx[7]] - coords[2, idx[3]], color='g')
#    quiver4 = ax.quiver(coords[0, idx[4]], coords[1, idx[4]], coords[2, idx[4]], coords[0, idx[8]] - coords[0, idx[4]], coords[1, idx[8]] - coords[1, idx[4]],coords[2, idx[8]] - coords[2, idx[4]], color='g')

    traceD.append([coords[0, 0], coords[1, 0], coords[2, 0]])
    line8.set_data(np.array(traceD)[:, 0:2].T)                                       # the trace of the drone body
    line8.set_3d_properties(np.array(traceD)[:, 2])

    traceP1.append([coords[0, 1], coords[1, 1], coords[2, 1]])                      # the trace of the propeller 1
    line9.set_data(np.array(traceP1)[:, 0:2].T)
    line9.set_3d_properties(np.array(traceP1)[:, 2])

    traceP2.append([coords[0, 2], coords[1, 2], coords[2, 2]])                     # the trace of the propeller 2
    line10.set_data(np.array(traceP2)[:, 0:2].T)
    line10.set_3d_properties(np.array(traceP2)[:, 2])

    traceP3.append([coords[0, 3], coords[1, 3], coords[2, 3]] )                    # the trace of the propeller 3
    line11.set_data(np.array(traceP3)[:, 0:2].T)
    line11.set_3d_properties(np.array(traceP3)[:, 2])

    traceP4.append([coords[0, 4], coords[1, 4], coords[2, 4]])                     # the trace of the propeller 4
    line12.set_data(np.array(traceP4)[:, 0:2].T)
    line12.set_3d_properties(np.array(traceP4)[:, 2])

# Set labels and legend
ax.set_xlabel('X-axis')
ax.set_ylabel('Y-axis')
ax.set_zlabel('Z-axis')

# Create the animation
animation = FuncAnimation(fig, update, frames=np.arange(t0, tf, 1 / fps), interval=2.5*fps, blit=False)
plt.close(fig)  # Prevents the final image from being displayed directly below the animation
# Show the plot
display(HTML(animation.to_jshtml()))
print(f'it took {time.time()-start0:.1f} seconds to run the code.')